In [ ]:
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [ ]:
import csv
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pickle

import torch
import hydra
import omegaconf
import pyrootutils
from tqdm import tqdm
from glob import glob

import matplotlib.pyplot as plt
from fgvc.data.plant_clef_data import PlantCLEFDataset, PlantSPECIESDataset
from torchmetrics.classification import MulticlassAccuracy, MulticlassPrecisionRecallCurve, MulticlassAveragePrecision
from torchmetrics.functional.classification import *

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("/home/ubuntu/FGVC11/data/PlantClef/PlantCLEFTrainLQ.csv",  delimiter=";", escapechar="/")

In [ ]:
# dataset class
cfg = omegaconf.OmegaConf.load("/home/ubuntu/FGVC11/configs/data/plant_clef_data.yaml")
datamodule = hydra.utils.instantiate(cfg)
datamodule.setup()

In [ ]:
cfg = omegaconf.OmegaConf.load("/home/ubuntu/FGVC11/configs/model/plant_clef_model.yaml")
model = hydra.utils.instantiate(cfg)
model.load_from_checkpoint("/home/ubuntu/FGVC11/logs/train/runs/test_edgenext_small_cross_entropy_specie_based_dataset_10/checkpoints/epoch_198.ckpt");
model.to("cuda");
model.eval();


In [ ]:
cats = datamodule.le.categories_[0]
test_targets = []
target_specie = []
test_preds = []
for batch in tqdm(datamodule.train_dataloader()):
    image = batch["image"].to("cuda")
    test_targets.append(batch["encoded_label"].cpu())
    target_specie.append(batch["label"].cpu())
    with torch.no_grad():
        output = torch.sigmoid(model(image))
    test_preds.append(output.cpu())
    # break
test_preds = torch.cat(test_preds)
test_targets = torch.cat(test_targets)
target_specie = torch.cat(target_specie)

In [ ]:
test_targets.shape, test_preds.shape, torch.argmax(test_targets, dim=1).shape

In [ ]:
cats = torch.tensor(cats)

In [ ]:
sum(cats[torch.argmax(test_targets, dim=1)] == target_specie)

In [ ]:
# mc_pr = MulticlassPrecisionRecallCurve(num_classes=len(cats), average="macro", thresholds=10)
# precision, recall, thresholds = mc_pr(test_preds, torch.argmax(test_targets, dim=1))
# # mc_pr.update(test_preds, torch.argmax(test_targets, dim=1))
# # fig_, ax_ = mc_pr.plot(score=True)

In [ ]:
# preds_single = []
# for i in range(len(test_preds)):
#     # keep the largest score and make zero for the rest of them
#     pred = torch.zeros_like(test_preds[i])
#     # assign the value
#     pred[torch.argmax(test_preds[i])] = test_preds[i][torch.argmax(test_preds[i])] 
#     preds_single.append(pred)
#     # break
# preds_single = torch.stack(preds_single)

In [ ]:
sum(torch.argmax(test_preds, dim=1) == torch.argmax(test_targets, dim=1))/len(test_targets)

In [ ]:
len(test_targets)

In [ ]:
10/78060

In [ ]:
ac = MulticlassAccuracy(num_classes=len(cats), average="macro")
ac(test_preds, torch.argmax(test_targets, dim=1))

In [ ]:
ap = MulticlassAveragePrecision(num_classes=len(cats), average="macro", thresholds=10)
ap(test_preds, torch.argmax(test_targets, dim=1))

In [ ]:
import timm

In [ ]:
model = timm.create_model("edgenext_base", pretrained=True, num_classes=7806)

In [ ]:
model